# Vacancy migration in graphene
## Exercise MMM 2025 - Week 3

In the present exercise, we will investigate the equilibrium structures of graphene as a periodic 2D lattice, as well as a vacancy (defect) diffusion mechanism. 
As a calculator we will use the code CP2K.
As an interaction potential we will use the Tersoff potential.
As minimization methods for minima and transition states (elastic band) we will use BFGS and the FIRE MD-based algorithm (https://www.math.uni-bielefeld.de/~gaehler/papers/fire.pdf).
There is a paper where the same diffusion mechanism is described with ab initio calculations: https://link.aps.org/doi/10.1103/PhysRevB.98.075439


### First part: graphene and defects - model preparation
A graphene periodic layer is prepared. Two visualization functions are defined.

In [ ]:
import numpy as np
from ase import Atoms
from ase.io import read
from ase.calculators.lammpsrun import LAMMPS
from ase.calculators.cp2k import CP2K
from ase.optimize import BFGS
from ase.build import graphene
import nglview as nv

# Graphene lattice


graph_0 = graphene(formula='C2',a=2.46,vacuum=1.0,size=(15,15,1))
print (len(graph_0)," Carbon atoms")



In [ ]:
def view_structure(structure,myvec=[]):
    """
    Use the ASE library to view an atoms object.

    Parameters
    ----------

    structure: Atoms object

    Returns
    -------

    NGLWidget with GUI: object to be viewed
    
    """
    t = nv.ASEStructure(structure)
    w = nv.NGLWidget(t, gui=True)
    w.add_unitcell()
    w.add_ball_and_stick()
    w.add_representation('label',label_type='atomindex',color='black')
    w.add_representation('spacefill',selection=myvec,color="blue",radius=0.5)
    return w

def view_trajectory(trajectory,myvec=[]):
    t2 = nv.ASETrajectory(trajectory)
    w2 = nv.NGLWidget(t2, gui=True)
    #w2.add_unitcell()
    w2.add_ball_and_stick()
    w2.add_representation('spacefill',selection=myvec,color="blue",radius=0.5)
    return w2

In [ ]:
from ase.io import write

In [ ]:
graph_0.set_pbc([True,True,False])
my = graph_0.get_cell()
my[2,2] = 30
graph_0.set_cell(my)
view_structure(graph_0)

A calculator based on CP2K is prepared

In [ ]:
inp="""&FORCE_EVAL
  &MM
    &FORCEFIELD
      &NONBONDED
        &TERSOFF
          A 1.3936E3
          ALPHA 0.00
          ATOMS C C
          B 3.467E2
          BETA 1.5724E-7
          BIGD 0.15
          BIGR 1.95
          C 3.8049E4
          D 4.384
          H -5.7058E-1
          LAMBDA1 3.4879
          LAMBDA2 2.2119
          LAMBDA3 0.0000
          N 7.2751E-1
        &END TERSOFF
      &END NONBONDED
      &SPLINE
        EPS_SPLINE 1.E-6
      &END SPLINE
    &END FORCEFIELD
    &POISSON
      &EWALD
        EWALD_TYPE none
      &END EWALD
    &END POISSON
    #&PRINT
    #  &NEIGHBOR_LISTS SILENT
    #  &END NEIGHBOR_LISTS
    #&END PRINT
  &END MM
&END FORCE_EVAL"""

In [ ]:
calc = CP2K(inp=inp,command="/usr/bin/cp2k_shell.psmp",poisson_solver='None',force_eval_method='FIST')

In [ ]:
graph_0.calc = calc


### Second part: using the defected graphene as initial and final state for diffusion.

An atom is removed from the two samples. Identify the atom in the two structures.

**ASSIGNMENT 1: Why do we need to reorder the positions in sample graph_b?**

In [ ]:
#
#  Vacancy in A
#

graph_a = graph_0.copy() 
graph_a.calc = calc



del graph_a[225]

#
#  Vacancy in B
#

graph_b = graph_0.copy()
graph_b.calc = calc

del graph_b[256]

view_structure(graph_a)


In [ ]:
#print (graph_a.get_positions()[225:256])

#
# Reorder
#
graph_b.positions[[255,225,226,227,228,229,230,231,232,233,234,235,236,237,238,239,240,241,242,243,244,245,246,247,248,249,250,251,252,253,254]]=graph_b.positions[[225,226,227,228,229,230,231,232,233,234,235,236,237,238,239,240,241,242,243,244,245,246,247,248,249,250,251,252,253,254,255]]
#print (graph_b.get_positions()[225:256])



In [ ]:
view_structure(graph_a)

In [ ]:
view_structure(graph_b)

### Fixing the atoms at the border of the vacancy region
**ASSIGNMENT 2: Insert different values of r_relax and imagine the situation for those values. Discuss the chosen value that will presumably lead to a good NEB convergence**


In [ ]:
#
#Apply Constraints
#

r_relax = INSERT A VALUE!!!
from ase.constraints import FixAtoms

my_ind_0 = [atom.index for atom in graph_0 if (graph_0.get_distance(atom.index,254,mic=True)>r_relax)]
c = FixAtoms(indices=my_ind_0)
graph_0.set_constraint(c)
graph_0.rattle(stdev=0.005)


my_ind_a = [atom.index for atom in graph_a if (graph_a.get_distance(atom.index,253,mic=True)>r_relax)]
c = FixAtoms(indices=my_ind_a)
graph_a.set_constraint(c)
graph_a.rattle(stdev=0.005)


my_ind_b = [atom.index for atom in graph_b if (graph_b.get_distance(atom.index,253,mic=True)>r_relax)]
c = FixAtoms(indices=my_ind_b)
graph_b.set_constraint(c)
graph_b.rattle(stdev=0.005)



view_structure(graph_a,my_ind_a)




### Let's optimize graphene first

In [ ]:

opt = BFGS(graph_0, trajectory='graphene_0_opt.traj')
opt.run(fmax=0.05)

In [ ]:
mytraj = read("graphene_0_opt.traj",":")
view_trajectory(mytraj,my_ind_0)

### Optimizing the defect

**Assignment 3: What happens if you uncomment the line #mypos[:,2] = 1.? Will something change in the final result of the optimization?**

In [ ]:
#
# Optimize the geometry of vacancy A
#
graph_a.rattle(stdev=0.05)
mypos = graph_a.get_positions()
#mypos[:,2] = 1.
graph_a.set_positions(mypos)
view_structure(graph_a)

opt = BFGS(graph_a, trajectory='graphene_a_opt.traj')
opt.run(fmax=0.05)
mytraj = read("graphene_a_opt.traj",":")
view_trajectory(mytraj,my_ind_a)


In [ ]:
print (graph_a.get_distance(194,224))
view_structure(graph_a)


In [ ]:
#
# Optimize the geometry of Vacancy B
#

graph_b.rattle(stdev=0.05)
mypos = graph_b.get_positions()
#mypos[:,2] = 1.
graph_b.set_positions(mypos)
opt = BFGS(graph_b, trajectory='graphene_b_opt.traj')
opt.run(fmax=0.05)
mytraj_b = read("graphene_b_opt.traj",":")


graph_a.write("graphene_a.xyz",format="extxyz")
graph_b.write("graphene_b.xyz",format="extxyz")
view_trajectory(mytraj_b,my_ind_b)

In [ ]:
print (graph_b.get_distance(257,226))
view_structure(graph_b,my_ind_b)

### Part 3: Nudged Elastic Band

Now, we can optimize the nudged elastic band. The initial and final configurations are the graphene with the vacancy in two different places (https://wiki.fysik.dtu.dk/ase/ase/neb.html)


**ASSIGNMENT 4: Try the NEB with 7 and then again with 11 replicas. Do we need a randomization of the coordinates (rattle)? Why? Which difference do you observe? Do you always arrive to convergence? Play with the parameters and comment**

In [ ]:
#
# NEB
#

n_replica = PUT_A_NUMBER_OF_REPLICAS
from ase import io
from ase.neb import NEB
from ase.optimize import MDMin, BFGS, FIRE, GPMin

# Read initial and final states:
initial = graph_a.copy()
final = graph_b.copy()



# Make a band consisting of n_replica images:
images = [initial]
images += [initial.copy() for i in range(n_replica - 2)]
images += [final]


#
# Restart
#


neb = NEB(images,k=0.05, climb=False)

# Interpolate linearly the positions of the three middle images:
neb.interpolate(apply_constraint = False)

for image in images [1:n_replica-1]:
       image.rattle(stdev=0.02)
    


# Set calculators:
      
for image in images[1:n_replica-1]:
    calc = CP2K(inp=inp,command="/usr/bin/cp2k_shell.psmp",poisson_solver='None',force_eval_method='FIST')
    image.calc = calc
    
# Optimize:
optimizer =  FIRE(neb, trajectory='A2B.traj')
optimizer.run(fmax=0.07)

**ASSIGNMENT 5: Observe the barrier and compare with the values in the paper. Discuss possible sources of errors in our approach**

In [ ]:

ene = np.zeros(n_replica)
i=0

for image in images[0:n_replica]:
    image.calc = calc    
    print (i,image.get_potential_energy())
    ene[i]=image.get_potential_energy()
    i = i+1
    
import matplotlib.pyplot as plt
plt.plot(ene)

plt.show()
    
#print (images[4].get_positions())


In [ ]:
view_trajectory(images[0:n_replica],my_ind_a)




### Part 4: Analysis of the structures 

**ASSIGNMENT 6: Discuss the corrugation plot as well as the geometry at the transition state**

In [ ]:
import matplotlib.pyplot as plt
from ase import Atoms

def plot_atoms_corrugation(atoms, index):
    """
    Plots the x and y coordinates of an ASE Atoms object in 2D, with z-encoded color.
    The plot maintains equal scaling for x and y axes, has a squared grid, and includes a colorbar.

    Args:
        atoms (ase.Atoms): The ASE Atoms object containing x, y, and z coordinates.

    Returns:
        None (displays the plot)
    """
    # Extract x, y, and z coordinates
    x_coords = atoms.positions[:, 0]
    y_coords = atoms.positions[:, 1]
    z_coords = atoms.positions[:, 2]

    # Calculate the aspect ratio
    L_x = max(x_coords) - min(x_coords)
    L_y = max(y_coords) - min(y_coords)
    aspect_ratio = L_y / L_x

    # Create a scatter plot with z-coordinates determining color
    fig, ax = plt.subplots(figsize=(8, 8 * aspect_ratio))  # Equal scaling for x and y axes
    scatter = ax.scatter(x_coords, y_coords, c=z_coords, cmap='viridis', s=50)  # Adjust 'cmap' and 's' as desired
    ax.set_xlabel('x')
    ax.set_ylabel('y')
    ax.set_title('Atoms Object: x-y Coordinates with Z-encoded Color '+str(index))
    plt.grid(True)
    plt.gca().set_aspect('equal')  # Set aspect ratio to be equal
    plt.colorbar(scatter, label='Z-coordinate')  # Add colorbar
    plt.show()



In [ ]:
import ipywidgets as ipw
from IPython.display import clear_output
output = ipw.Output()
display(output)

In [ ]:
# Example usage:
import time
i = 0
for image in images[0:n_replica]:  
    time.sleep(1)
    with output:
        clear_output()
        plot_atoms_corrugation(image,i)
    i = i+1


In [ ]:
plot_atoms_corrugation(images[INSERT_HERE_THE_TRANSITION_STATE_REPLICA],6)

In [ ]:
view_structure(images[INSERT_HERE_THE_CORRECT_TRANSITION_STATE_REPLICA],my_ind_a)